In [30]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv,Series, DataFrame
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas.io.data as web
import datetime
import tensorflow as tf
from sklearn import svm, preprocessing
import matplotlib.pyplot as plt
import math
np.set_printoptions(threshold=np.inf)



In [2]:
class KOSPIDATA:

    def __init__(self):
        start = datetime.datetime(1998, 5, 1)
        end = datetime.datetime(2016, 12, 31)
        kospi = web.DataReader("^KS11", "yahoo", start, end)

        self.arr_date= np.array(kospi.index)
        self.arr_open = np.array(kospi['Open'], dtype=float)
        self.arr_close= np.array(kospi['Adj Close'], dtype=float)
        self.arr_high= np.array(kospi.High, dtype=float)
        self.arr_low= np.array(kospi.Low, dtype=float)
        self.arr_volume= np.array(kospi.Volume, dtype=float)

In [4]:
def create_dataset(dataset, look_back=3):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        
        if dataset[i+look_back] > dataset[i+look_back-1]:    
            dataY.append(1)
        elif dataset[i+look_back] < dataset[i+look_back-1]:    
            dataY.append(0)
    return np.array(dataX), np.array(dataY)

In [5]:
K = KOSPIDATA()

# FEATURES = ['high', 'low', 'open', 'close', 'volume']
FEATURES = ['high', 'low', 'open', 'close']

data = {'year': K.arr_date,
        'open': K.arr_open,
        'high': K.arr_high,
        'low': K.arr_low,
        'close': K.arr_close}

# df = DataFrame(data, columns=['year', 'high', 'low', 'open', 'close'])
df = DataFrame(data, columns=['year','close'])

In [6]:
dataset = df['close'].values

In [7]:
np.random.seed(7)

In [8]:
# dataset = dataset.reshape(-1,1)

In [9]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

//anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [10]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

In [11]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [36]:
scaler.inverse_transform(trainX)

array([[  391.799988,   376.230011,   379.470001],
       [  376.230011,   379.470001,   373.040009],
       [  379.470001,   373.040009,   361.579987],
       [  373.040009,   361.579987,   351.859985],
       [  361.579987,   351.859985,   356.579987],
       [  351.859985,   356.579987,   362.940002],
       [  356.579987,   362.940002,   352.829987],
       [  362.940002,   352.829987,   359.      ],
       [  352.829987,   359.      ,   362.529999],
       [  359.      ,   362.529999,   369.170013],
       [  362.529999,   369.170013,   362.299988],
       [  369.170013,   362.299988,   353.670013],
       [  362.299988,   353.670013,   331.899994],
       [  353.670013,   331.899994,   311.98999 ],
       [  331.899994,   311.98999 ,   313.480011],
       [  311.98999 ,   313.480011,   323.089996],
       [  313.480011,   323.089996,   336.630005],
       [  323.089996,   336.630005,   336.700012],
       [  336.630005,   336.700012,   324.100006],
       [  336.700012,   324.100

In [37]:
trainX.shape

(3090, 3)

In [35]:
trainY.shape

(3085,)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [18]:
model = Sequential()
model.add(LSTM(4, input_dim=look_back))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, nb_epoch=100, batch_size=1, verbose=2)

ValueError: Error when checking model input: expected lstm_input_1 to have 3 dimensions, but got array with shape (3090, 3)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
testPredict.shape

In [ ]:
trainPredict

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY)

In [ ]:
trainScore = math.sqrt(((trainY[0] - trainPredict[:,0]) ** 2).mean(axis=None))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(((testY[0] - testPredict[:,0]) ** 2).mean(axis=None))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting

In [ ]:
trainPredictPlot = np.zeros_like(dataset)
trainPredictPlot[look_back:len(trainPredict)+look_back] = trainPredict[:,0]

In [ ]:
testPredictPlot = np.zeros_like(dataset)
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1] = testPredict[:,0]

In [ ]:
plt.plot(scaler.inverse_transform(dataset))

In [ ]:
plt.plot(trainPredictPlot)

In [ ]:
plt.plot(testPredictPlot)

In [ ]:
plt.show()

trainPredictPlot = np.empty_like(dataset)
print(trainPredictPlot.shape)
trainPredictPlot[:] = np.nan
print(trainPredictPlot.shape)
trainPredictPlot[look_back:len(trainPredict)+look_back] = trainPredict[:,0]
# shift test predictions for plotting